In [1]:
import random
import itertools

import pandas

In [ ]:
DATA_PATH: str = "../../../data"
DATASET_PATH: str = f"{DATA_PATH}/processed/twitter.english.dataset.enriched.csv"

In [ ]:
data = (
    pandas.read_csv(DATASET_PATH, index_col=[0])
    .pipe(
        lambda _df: _df[_df["author_id_reply"].isin(
            (
                (data.groupby("author_id_reply").size() > 3)
                .pipe(lambda _srs: _srs[_srs])
            ).index
        )]
        )
)
display(len(data))
data.head()

In [ ]:
positive_samples = {
    author: [
        sample for sample in itertools.combinations(group["text_post"].tolist(), 3)
    ]
    for author, group in data.groupby("author_id_reply")
}
sum(len(group) for group in positive_samples.values())

In [ ]:
negative_samples = {
    author: [
        (
            *random.choice(positive_samples[author])[:2],
            random.choice(
                positive_samples[
                    random.choice(list(set(positive_samples.keys() - set([author]))))
                ]
            )[2],
        )
        for _ in range(450)
    ]
    for author in set(positive_samples.keys())
}
sum(len(group) for group in negative_samples.values())

In [ ]:
dataset = pandas.concat(
    [
        pandas.DataFrame(
            [
                dict(
                    history_1=sample[0],
                    history_2=sample[1],
                    stimulus=sample[2],
                    action=label,
                )
                for group in sets
                for sample in group
            ]
        ).sample(150_000)
        for label, sets in [
            (True, positive_samples.values()),
            (False, negative_samples.values()),
        ]
    ]
)

dataset.to_csv(DATASET_PATH.replace("enriched", "decision"))
len(dataset)